In [ ]:
# Import modules
import torch
import torch.nn as nn
import torch.optim as optim
import pandas as pd
import numpy as np
from torch.utils.data import DataLoader
from torch.utils.data import TensorDataset
from sklearn.model_selection import train_test_split, StratifiedShuffleSplit
from sklearn.preprocessing import StandardScaler

In [ ]:
# Load the dataset from CSV file
data = pd.read_csv('Crop_recommendation.csv')

In [ ]:
# Assign features and labels
features = data[['N','P','K','temperature','humidity','ph','rainfall']].values
labels = data['label'].values

In [ ]:
# Normalize input features
scaler = StandardScaler()
features_normalized = scaler.fit_transform(features)

In [ ]:
# Convert label strings to numeric representations
label_to_index = {label: idx for idx, label in enumerate(np.unique(labels))}
numeric_labels = np.array([label_to_index[label] for label in labels])

In [ ]:
# Convert labels to torch.long data type
labels = torch.tensor(numeric_labels, dtype=torch.long)

In [ ]:
# Initialize StratifiedShuffleSplit
sss = StratifiedShuffleSplit(n_splits=1, test_size=0.3, random_state=42)

# Split the data into training and test sets while preserving label distribution
for train_index, test_index in sss.split(features_normalized, labels):
    X_train, X_test = features_normalized[train_index], features_normalized[test_index]
    y_train, y_test = labels[train_index], labels[test_index]

In [ ]:
# Convert features to float32 and labels to int64
X_train = torch.tensor(X_train, dtype=torch.float32)
y_train = y_train.clone().detach()
X_test = torch.tensor(X_test, dtype=torch.float32)
y_test = y_test.clone().detach()

In [ ]:
# Create TensorDataset objects
train_dataset = TensorDataset(X_train, y_train)
test_dataset = TensorDataset(X_test, y_test)

In [ ]:
# Define data loaders
train_loader = DataLoader(train_dataset, batch_size=100, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=100, shuffle=True)

In [ ]:
# Define neural network architecture with 2 hidden layers
class NeuralNetwork2Hidden(nn.Module):
    def __init__(self, activation, n_neurons, dropout_rate, weight_init):
        super().__init__()
        self.fc1 = nn.Linear(7, n_neurons)
        self.bn1 = nn.BatchNorm1d(n_neurons)
        self.fc2 = nn.Linear(n_neurons, n_neurons)
        self.bn2 = nn.BatchNorm1d(n_neurons)
        self.fc3 = nn.Linear(n_neurons, 22)
        self.activation = activation()
        self.dropout = nn.Dropout(dropout_rate)
        self.weight_init = weight_init
        self.initialize_weights()

    def initialize_weights(self):
        self.weight_init(self.fc1.weight)
        self.weight_init(self.fc2.weight)
        self.weight_init(self.fc3.weight)

    def forward(self, x):
        x = self.activation(self.bn1(self.fc1(x)))
        x = self.dropout(x)
        x = self.activation(self.bn2(self.fc2(x)))
        x = self.dropout(x)
        x = self.fc3(x)
        return x

# Define neural network architecture with 5 hidden layers
class NeuralNetwork5Hidden(nn.Module):
    def __init__(self, activation, n_neurons, dropout_rate, weight_init):
        super().__init__()
        self.fc1 = nn.Linear(7, n_neurons)
        self.bn1 = nn.BatchNorm1d(n_neurons)
        self.fc2 = nn.Linear(n_neurons, n_neurons)
        self.bn2 = nn.BatchNorm1d(n_neurons)
        self.fc3 = nn.Linear(n_neurons, n_neurons)
        self.bn3 = nn.BatchNorm1d(n_neurons)
        self.fc4 = nn.Linear(n_neurons, n_neurons)
        self.bn4 = nn.BatchNorm1d(n_neurons)
        self.fc5 = nn.Linear(n_neurons, n_neurons)
        self.bn5 = nn.BatchNorm1d(n_neurons)
        self.fc6 = nn.Linear(n_neurons, 22)
        self.activation = activation()
        self.dropout = nn.Dropout(dropout_rate)
        self.weight_init = weight_init
        self.initialize_weights()

    def initialize_weights(self):
        self.weight_init(self.fc1.weight)
        self.weight_init(self.fc2.weight)
        self.weight_init(self.fc3.weight)
        self.weight_init(self.fc4.weight)
        self.weight_init(self.fc5.weight)
        self.weight_init(self.fc6.weight)

    def forward(self, x):
        x = self.activation(self.bn1(self.fc1(x)))
        x = self.dropout(x)
        x = self.activation(self.bn2(self.fc2(x)))
        x = self.dropout(x)
        x = self.activation(self.bn3(self.fc3(x)))
        x = self.dropout(x)
        x = self.activation(self.bn4(self.fc4(x)))
        x = self.dropout(x)
        x = self.activation(self.bn5(self.fc5(x)))
        x = self.dropout(x)
        x = self.fc6(x)
        return x

In [ ]:
# Define the training function for both models
def train_model(model, train_loader, optimizer, criterion, epochs):
    for epoch in range(epochs):
        model.train()
        for batch_idx, (data, target) in enumerate(train_loader):
            optimizer.zero_grad()
            output = model(data)
            loss = criterion(output, target)
            loss.backward()
            optimizer.step()
            if batch_idx % 100 == 0:
                print('Train Epoch: {} [{}/{} ({:.0f}%)]\tLoss: {:.6f}'.format(
                    epoch, batch_idx * len(data), len(train_loader.dataset),
                    100. * batch_idx / len(train_loader), loss.item()))

In [ ]:
# Define parameters for training NeuralNetwork2Hidden
activation_2hidden = nn.LeakyReLU
n_neurons_2hidden = 40
dropout_rate_2hidden = 0.2
weight_init_2hidden = nn.init.xavier_normal_
optimizer_2hidden = optim.Adamax
learning_rate_2hidden = 0.01
criterion_2hidden = nn.CrossEntropyLoss

In [ ]:
# Training NeuralNetwork2Hidden
model_2hidden = NeuralNetwork2Hidden(activation_2hidden, n_neurons_2hidden, dropout_rate_2hidden, weight_init_2hidden)
optimizer_2hidden = optimizer_2hidden(model_2hidden.parameters(), lr=learning_rate_2hidden)
criterion_2hidden = criterion_2hidden()
train_model(model_2hidden, train_loader, optimizer_2hidden, criterion_2hidden, epochs=100)

In [ ]:
# Define parameters for training NeuralNetwork5Hidden
activation_5hidden = nn.LeakyReLU
n_neurons_5hidden = 40
dropout_rate_5hidden = 0.0
weight_init_5hidden = nn.init.xavier_normal_
optimizer_5hidden = optim.Adam
learning_rate_5hidden = 0.01
criterion_5hidden = nn.CrossEntropyLoss

In [ ]:
# Training NeuralNetwork5Hidden
model_5hidden = NeuralNetwork5Hidden(activation_5hidden, n_neurons_5hidden, dropout_rate_5hidden, weight_init_5hidden)
optimizer_5hidden = optimizer_5hidden(model_5hidden.parameters(), lr=learning_rate_5hidden)
criterion_5hidden = criterion_5hidden()
train_model(model_5hidden, train_loader, optimizer_5hidden, criterion_5hidden, epochs=100)

In [ ]:
# Save trained models to disk
torch.save(model_2hidden.state_dict(), 'model_2hidden.pth')
torch.save(model_5hidden.state_dict(), 'model_5hidden.pth')